# Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Connecting to Box in order to load the DMD data videos

In [ ]:
!pip install boxsdk

In [ ]:
from boxsdk import OAuth2, Client
oauth = OAuth2(
  client_id='ob2zdfkdzs8pevzi5irt10enk1wkog7w',
  client_secret='JFP7cmpSJOgLbUv5FiLxzBlog9xEcJJp',
  access_token='jen0IY1rsyV3bm8pdRAinHy1GCiOnRHH',
)
client = Client(oauth)

items = client.folder('0').get_items()
for item in items:
    print(f'{item.type.capitalize()} - {item.name}')

Folder - DMD - Driver Monitoring Dataset
Folder - shared_folder
File - Get Started with Box.pdf


## Checking the folder object and get_items method

In [ ]:
folder_id = '126765327560' # This is the folder id for the

folder = client.folder(folder_id).get()

items = folder.get_items()
for item in items:
    print(f'{item.type.capitalize()} - {item.name}')

Folder - gA
Folder - gB
Folder - gC
Folder - gE
Folder - gF
Folder - gZ


## Using .Content()

In [ ]:
# Import the required exception from the Box SDK
from boxsdk.exception import BoxOAuthException
import os
import requests

# Define the function to download files
def download_files(folder_id, gdrive_folder, count=0):
    # Get the items in the folder
    items = client.folder(folder_id).get_items()
    for item in items:
        try:
            # If the item is a file, its name contains 'rgb', and it's an .mp4 video, download it
            if item.type == 'file' and 'rgb' in item.name and item.name.endswith('.mp4'):
                file_path = os.path.join(gdrive_folder, item.name)
                # Check if the file already exists in the destination folder
                if not os.path.exists(file_path):
                    print(f'Downloading file {item.name}...')
                    # Use the Box SDK to download the file content
                    file_content = client.file(item.id).content()
                    # Write the file content to a file in the Google Drive folder
                    with open(file_path, 'wb') as f:
                        f.write(file_content)
                    # Increment the counter
                    count += 1
                else:
                    print(f'File {item.name} already exists.')
                # If 50 files have been downloaded, stop
                if count >= 2:
                    return count
            # If the item is a folder, recurse into it
            elif item.type == 'folder':
                count = download_files(item.id, gdrive_folder, count)
                if count >= 2:
                    return count
        # If the token has expired, print a message
        except BoxOAuthException:
            print("Token expired. Please refresh the token and try again.")
    return count



In [ ]:
# Start the download process
# root folder ID could be extracted from the url https://app.box.com/folder/126765327560?s=h9k8q2uq7vx2yvnsggtsnt7vgvs9m6ol
root_folder_id = '126765327560'
google_drive_folder = '/content/drive/MyDrive/Test'
download_files(root_folder_id,google_drive_folder)

# Creating a new dataset of paired images with degredation

In [ ]:
import cv2
import os
from PIL import Image, ImageEnhance

# Function to extract frames
def extract_frames(video_path, rate, img_name, target_path):
    vidcap = cv2.VideoCapture(video_path)
    count = 0
    success = True
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC, (count * 1000 / rate))  # Extract 'rate' frame every  second
        success, image = vidcap.read()
        if success:
            cv2.imwrite(os.path.join(target_path, f"{img_name}{count}.png"), image)
        count += 1

# Function to degrade images
def degrade_images(target_path, source_path, exposure_values):
    for filename in os.listdir(target_path):
        if filename.endswith(".png"):
            img = Image.open(os.path.join(target_path, filename))
            enhancer = ImageEnhance.Brightness(img)
            for ev in exposure_values:
                new_img = enhancer.enhance(1 + ev)
                new_img.save(os.path.join(source_path, f"{filename[:-4]}-E{'P' if ev >= 0 else 'N'}{abs(ev)}.png"))

# Path to your Google Drive folder containing the videos
drive_path = "/content/drive/MyDrive/DMD-Database-RGBMP4-Videos"

# Create 'source' and 'target' directories
source_path = os.path.join(drive_path, "source")
target_path = os.path.join(drive_path, "target")
os.makedirs(source_path, exist_ok=True)
os.makedirs(target_path, exist_ok=True)

# Extract frames from videos
for filename in os.listdir(drive_path):
    if filename.endswith(".mp4"):  # Assuming the videos are in .mp4 format
        video_path = os.path.join(drive_path, filename)
        extract_frames(video_path, rate=0.1, img_name=filename[:-4], target_path=target_path)  # Change rate as needed

# Degrade images
exposure_values = [0, +1, -0.2, +1.5, -0.3]
degrade_images(target_path, source_path, exposure_values)
